# Train an IntegratedML model on ED Readmit likelihood Dataset

**NOTE: This Notebook will not run as-is!**

This notebook requires:
- External data (Data.PatientCostData table)
- Custom connection script (Initializations/Conns.py)

To adapt for the DB-API driver, replace the connection setup with:
```python
import iris
conn = iris.connect("irisimlsvr:1972/USER", "SUPERUSER", "SYS")
curs = conn.cursor()
```

This Notebook demonstrates:
- Creating views to segment data into training and test sets
- Defining and training an IntegratedML model to predict ED Readmits in the next 90 days
- Comparing the resulting model's predictions to data in the test set
- Using the IntegratedML "VALIDATE MODEL" command to calculate accuracy metrics

### 1. Get connection and cursor

In [ ]:
import pandas as pd

In [ ]:
%run -i '../Initializations/Conns.py'

### 2. Create and specify the source data table(s)

In [ ]:
#Use this block to create a starting data set that you can/will build upon.
#NOTE: It is always useful to have a unique identifier in the data
TargetTable = 'Data.EDEncsPredB90View'
TrainTable = 'Data.EDEncsTraining'
TestTable = 'Data.EDEncsTesting'

### 3. Review the data to ensure the Target variable and Independent variables are in good standing.

In [ ]:
tKeep()
import pandas as pd
from IPython.display import display

df = pd.read_sql("select top 3 * from Data.PatientCostData", iconn)
print(df)
print(df.columns)

### Drop and unwanted fields

In [ ]:
df = df.drop(['T30', 'T60', 'T90'], axis = 1)
Usable = str(list(df.columns)).replace("', '", ",")[2:-2]
Usable

In [ ]:
icurs.execute(' \
    create or replace view %s as \
        select case when t90 > 0 then 1 else 0 end as B90, %s \
        from Data.PatientCostData' % (TargetTable, Usable))
df1 = pd.read_sql('SELECT COUNT(*) as Recs FROM %s' % TargetTable, iconn)
TargetVar = 'B90'
display(df1)

In [ ]:
Distro = pd.read_sql('select %s, count(*) as Recs from %s group by %s' % (TargetVar, TargetTable, TargetVar), iconn)
Distro

### 4. Assess the probability of your target and sample accordingly into split training and testing datasets

In [ ]:
#we want to split the data into Training (80%) and Test (20%), ...
# but also reduce the ratio of Negative (ED Enc = 0) to Positive
Train = 0.8
TVRatio = 2
PT_List = pd.read_sql('select DRID, %s from %s order by %s, DRID' % (TargetVar, TargetTable, TargetVar), iconn)
PT_List.index = PT_List['DRID']

In [ ]:
#Create the lists, check the ratios, and create the "In lists":
TrainList = PT_List[PT_List[TargetVar] == 0].sample(int(Distro['Recs'].loc[1]*TVRatio*Train)) \
    .append(PT_List[PT_List[TargetVar] == 1].sample(int(Distro['Recs'].loc[1]*Train)))
TrainList['Flag'] = 1
TrainList.index = TrainList['DRID']
print(TrainList.pivot_table(index = TargetVar, values = 'DRID', aggfunc = 'count'))
#NOTE: It is IMPERATIVE that Test does NOT contain any Train data
TestList = PT_List.join(TrainList['Flag'], how = 'left')
TestList = TestList[(TestList['Flag'] != 1)]
TestList = TestList[(TestList[TargetVar] == 1)].append(TestList[TestList[TargetVar] == 0].sample(int(len(TestList)*0.25)))
print(TestList.pivot_table(index = TargetVar, values = 'DRID', aggfunc = 'count'))
TrainIns = str(list(TrainList['DRID']))[1:-1]
TestIns = str(list(TestList['DRID']))[1:-1]

In [ ]:
# Training set view
icurs.execute(''' \
    CREATE or replace VIEW %s AS \
        SELECT * FROM %s 
        WHERE DRID in (%s)''' \
    % (TrainTable, TargetTable, TrainIns))
# Prediction set
icurs.execute(''' \
    CREATE or replace VIEW %s AS \
        SELECT * FROM %s 
        WHERE DRID in (%s)''' \
    % (TestTable, TargetTable, TestIns))

### 6. Create and Train an IntegratedML Model using default settings
IntegratedML only needs a model name, the name of the column that is the target column to predict, and a table (or SELECT query to specify input columns.

In [ ]:
try:
    icurs.execute("CREATE MODEL NewEncModel PREDICTING (%s) FROM %s" % (TargetVar, TrainTable))
except:
    icurs.execute("DROP MODEL NewEncModel")
    icurs.execute("CREATE MODEL NewEncModel PREDICTING (%s) FROM %s" % (TargetVar, TrainTable))

Now that the model is defined, you can TRAIN it, which invokes the AutoML machine learning procedure.

In [ ]:
icurs.execute("set ml configuration %AutoML")
icurs.execute("TRAIN MODEL NewEncModel as NewEncModel_Auto")
icurs.execute("set ml configuration DRCfg")
icurs.execute("TRAIN MODEL NewEncModel as NewEncModel_DR")

Once that finishes, you can see some information about the model in the "ML_TRAINED_MODELS" table.

In [ ]:
pd.read_sql("SELECT * FROM INFORMATION_SCHEMA.ML_TRAINED_MODELS", iconn)

### 7. Compare model output to data it has not seen yet
Now you can use SQL to SELECT data from another table, run the IntegratedML model on this new data, and see how well the predictions match the data!

In [ ]:
TestSet = pd.read_sql('''
    SELECT PREDICT(NewEncModel use NewEncModel_Auto) AS PredictedEncs,
        case when B90 = 1 then 1 end AS ActualPos,
        case when B90 != 1 then 0 end AS ActualNeg
    FROM %s''' % (TestTable), iconn)

In [ ]:
print(pd.pivot_table(TestSet, index = 'PredictedEncs', values = ['ActualPos', 'ActualNeg'], aggfunc = 'count'))
print('Accuracy: '+str(round((len(TestSet[(TestSet['PredictedEncs'] == TestSet['ActualPos']) \
            | (TestSet['PredictedEncs'] == TestSet['ActualNeg'])])/len(TestSet))*100))+'%')
print('Misclassification Rate: '+str(round((len(TestSet[(TestSet['PredictedEncs'] != TestSet['ActualPos']) \
            & (TestSet['PredictedEncs'] != TestSet['ActualNeg'])])/len(TestSet))*100))+'%')
print('%FP: '+str(round((len(TestSet[(TestSet['PredictedEncs'] == 1) & (TestSet['ActualNeg'] == 0)])/ \
            len(TestSet[TestSet['ActualNeg'] == 0]))*100))+'%')
print('%FN: '+str(round((len(TestSet[(TestSet['PredictedEncs'] == 0) & (TestSet['ActualPos'] == 1)])/ \
            len(TestSet[TestSet['ActualPos'] == 1]))*100))+'%')

### 8. VALIDATE MODEL command calculates accuracy metrics
You can certainly take that output above and calculate the accuracy using a standard formula, but IntegratedML has a built-in function to do that!

Each time you run the command "VALIDATE MODEL..." it generates a set of metrics calculated on the data passed into the query. Since this table can be a bit difficult to read in its raw form we use a simple "pivot" call to arrange the data.

In [ ]:
icurs.execute("VALIDATE model NewEncModel use NewEncModel_Auto FROM Data.EDEncsTesting")
#df5 = pd.read_sql("SELECT * FROM INFORMATION_SCHEMA.ML_VALIDATION_METRICS", iconn)
#df5
#df6 = df5.pivot(index='VALIDATION_RUN_NAME', columns='METRIC_NAME', values='METRIC_VALUE')
#display(df6)